## Agenda
- 리인덱싱
- 삭제
- 연산
- 정렬

In [2]:
import pandas as pd
import numpy as np

## 1.2 핵심 기능

### 1.2.1 리인덱싱

#### 시리즈 리인덱싱

In [3]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

- 새로운 인덱스가 추가되면 `NaN`이 사용

In [7]:
# reindex 를 통해서 넘버링을 다시 해줄수 있다.
# 새로운 인덱스 추가 가능, 순서 변경 가능 
obj = obj.reindex(["a", "b", "c", "d", "e"])
obj

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

- 지정되지 않은 인덱스는 무시

In [8]:
obj.reindex(["a", "b", "c"])

a   -5.3
b    7.2
c    3.6
dtype: float64

#### 결측치 채우기 1: `method` 키워드 인자

- 리인덱싱 과정에서 결측치가 발생할 때 여러 방식으로 채울 수 있다.
- `method='fill'` 키워드 인자는 결측치를 위쪽에 위치한 값으로 채운다.

__주의사항:__ 인덱스가 오름 또는 내림 차순으로 정렬되어 있는 경우에만 가능하다.

In [9]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 5])
obj3

0      blue
2    purple
5    yellow
dtype: object

In [10]:
obj3.reindex(range(6))

0      blue
1       NaN
2    purple
3       NaN
4       NaN
5    yellow
dtype: object

- 아랫쪽에 있는 값으로 채울 수도 있다.
- `method='bfill'`

In [11]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html#pandas.DataFrame.reindex
obj3.reindex(range(6), method = "bfill")

0      blue
1    purple
2    purple
3    yellow
4    yellow
5    yellow
dtype: object

In [14]:
# ffill 앞쪽에 있는 값ㅇ로 채워 준다
obj3.reindex(range(6), method = "ffill")


0      blue
1      blue
2    purple
3    purple
4    purple
5    yellow
dtype: object

- 아니면 가장 가까운 곳에 있는 값으로 채울 수도 있다.
- `method='nearest'`

In [13]:
# 아랫값을 기준으로 채워진다. 
obj3.reindex(range(6), method = "nearest")

0      blue
1    purple
2    purple
3    purple
4    yellow
5    yellow
dtype: object

#### 결측치 채우기 2: `fill_value` 키워드 인자

- 리인덱싱 과정에서 발생하는 모든 결측치를 지정된 값으로 대체
- 기본값은 `NaN` 이다.

- 리인덱싱은 항상 새로운 시리즈를 생성한다.
- 따라서 `obj3` 자체는 변하지 않는다.

In [15]:
obj3.reindex(range(6), fill_value="Color")

0      blue
1     Color
2    purple
3     Color
4     Color
5    yellow
dtype: object

In [16]:
obj3.reindex(range(6), fill_value="Color", inplce=True) # 참고 inplace 가 없다. 

TypeError: Series.reindex() got an unexpected keyword argument 'inplce'

#### 데이터프레임 리인덱싱

- 데이터프레임은 `index`와 `columns` 속성에 대해 리인덱싱이 가능하며 작동법은 Series의 인덱싱과 동일하다.

In [17]:
frame = pd.DataFrame(
    np.arange(9).reshape((3,3)),
    index = ["a", "c", "d"],
    columns = ["Ohio", "Texas", "California"]
)

In [19]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


- `reindex()` 메서드는 기본적으로 행의 `index` 에 대해 작동

In [20]:
frame.reindex(["a", "d", "c"])

,Ohio,Texas,California
a,0,1,2
d,6,7,8
c,3,4,5


- 열의 `columns`에 대해서는 `columns` 키워드 인자를 활용

In [22]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [21]:
states = ['Texas', 'Utah', 'California'] # Utah는 존재하지 않으므로 NaN으로 채워진다. 
states2 = ['Texas', 'California', 'Ohio']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [23]:
frame.reindex(columns=states2)

,Texas,California,Ohio
a,1,2,0
c,4,5,3
d,7,8,6


In [24]:
frame.reindex?
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html

Signature:
frame.reindex(
    labels=None,
    *,
    index=None,
    columns=None,
    axis: 'Axis | None' = None,
    method: 'ReindexMethod | None' = None,
    copy: 'bool | None' = None,
    level: 'Level | None' = None,
    fill_value: 'Scalar | None' = nan,
    limit: 'int | None' = None,
    tolerance=None,
) -> 'DataFrame'
Docstring:
Conform DataFrame to new index with optional filling logic.

Places NA/NaN in locations having no value in the previous index. A new object
is produced unless the new index is equivalent to the current one and
``copy=False``.

Parameters
----------

labels : array-like, optional
    New labels / index to conform the axis specified by 'axis' to.
index : array-like, optional
    New labels for the index. Preferably an Index object to avoid
    duplicating data.
columns : array-like, optional
    New labels for the columns. Preferably an Index object to avoid
    duplicating data.
axis : int or str, optional
    Axis to target. Can be either the axis 

### 5.2.2 `drop()` 메서드

- 특정 행 또는 열의 인덱스를 제외한 나머지로 이루어진 시리즈/데이터프레임을 구할 때 사용
- 시리즈의 경우 인덱스를 한 개 또는 여러 개 지정하면 나머지로 이루어진 시리즈를 얻을수 있다. 

In [25]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [26]:
obj.drop("c")

a    0.0
b    1.0
d    3.0
dtype: float64

In [29]:
del obj.loc["c"]

AttributeError: __delitem__

- `inplace=True` 키워드 인자를 이용하여 원본 수정 가능
- 사용시 주의 

In [31]:
obj.drop(["c", "d"], inplace = True)

In [32]:
obj

a    0.0
b    1.0
dtype: float64

- 데이터프레임의 경우도 기본적으로 행의 인덱스를 기준으로 작동한다.

In [33]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [34]:
data.drop(['Ohio', 'Colorado'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


- 열을 기준으로 작동하게 하려면 `axis=1`로 지정한다.

In [37]:
# data.drop(['one']) #axis =0 가 기본값이기 때문에 찾지 못한다. 

In [35]:
data.drop(['one'], axis = 1)

,two,three,four
Ohio,1,2,3
Colorado,5,6,7
Utah,9,10,11
New York,13,14,15


- `axis='columns'` 지정

In [41]:
data.drop(['one'], axis = "columns")

,two,three,four
Ohio,1,2,3
Colorado,5,6,7
Utah,9,10,11
New York,13,14,15


In [42]:
data.drop(['Colorado'], axis = "rows")

,one,two,three,four
Ohio,0,1,2,3
Utah,8,9,10,11
New York,12,13,14,15


In [46]:
frame.reindex(columns=states).dropna(axis=0)

,Texas,Utah,California


In [47]:
frame.reindex(columns=states).dropna(axis=1)

,Texas,California
a,1,2
c,4,5
d,7,8


### 1.2.3 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

#### 시리즈의 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

시리즈의 경우 1차원 넘파이 어레이와 거의 동일하게 작동한다.
다만 정수 대신에 지정된 인덱스를 사용할 때 조금 차이가 있다.

In [2]:
import numpy as np
import pandas as pd

In [3]:
obj = pd.Series(np.arange(4), index = ["a", "b", "c", "d"])

In [4]:
obj

a    0
b    1
c    2
d    3
dtype: int32

In [5]:
obj.b

1

In [6]:
obj["b"]

1

In [7]:
obj.loc["b"]

1

In [8]:
obj.iloc[1]

1

In [9]:
obj[1]

C:\Users\ohminy\AppData\Local\Temp\ipykernel_19444\2469632899.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[1]


1

In [11]:
obj[1:3]

b    1
c    2
dtype: int32

In [12]:
obj.iloc[[1,3,2]]

b    1
d    3
c    2
dtype: int32

In [13]:
obj[[1,3,2]]

C:\Users\ohminy\AppData\Local\Temp\ipykernel_19444\2900159710.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[[1,3,2]]


b    1
d    3
c    2
dtype: int32

- 여러 개의 인덱스를 리스트로 지정하여 인덱싱을 진행

In [14]:
obj.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [15]:
obj.loc[["b", "a", "d"]]

b    1
a    0
d    3
dtype: int32

- 필터링(부울 인덱싱)은 동일하게 작동

In [18]:
obj[obj > 1]

c    2
d    3
dtype: int32

- Label 슬라이싱

In [20]:
obj["b":"d"]

b    1
c    2
d    3
dtype: int32

In [21]:
obj1_1 = obj.loc[["b", "a", "d"]]

In [24]:
obj1_1["b":"d"] = 10 

In [25]:
obj1_1

b    10
a    10
d    10
dtype: int32

__주의사항:__ 
- 라벨 슬라이싱은 기본적으로 알파벳 순서를 따르며시리즈에 사용된 순서와 상관 없다.

In [27]:
obj2 = obj.reindex(["b", "d", "c", "a"])

In [28]:
obj2

b    1
d    3
c    2
a    0
dtype: int32

In [30]:
obj2["b":"c"]

b    1
d    3
c    2
dtype: int32

In [31]:
obj2["a":"c"]

Series([], dtype: int32)

#### 데이터프레임의 인덱싱, 슬라이싱, 필터링(부울 인덱싱)

In [32]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index = ["Ohio", "Colorado", "Utah", "New_York"],
                    columns= ["one", "two", "three", "four"]
                   )

In [33]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New_York,12,13,14,15


In [34]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New_York    13
Name: two, dtype: int32

In [35]:
data[["two", "one"]]

,two,one
Ohio,1,0
Colorado,5,4
Utah,9,8
New_York,13,12


In [40]:
type(data["two"])

pandas.core.series.Series

In [41]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New_York    13
Name: two, dtype: int32

In [38]:
type(data[["two"]])

pandas.core.frame.DataFrame

In [39]:
data[["two"]]

,two
Ohio,1
Colorado,5
Utah,9
New_York,13


In [42]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [46]:
mask = data["three"] > 2

In [48]:
mask

Ohio        False
Colorado     True
Utah         True
New_York     True
Name: three, dtype: bool

In [50]:
data[mask]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New_York,12,13,14,15


In [52]:
data["three"][mask]

Colorado     6
Utah        10
New_York    14
Name: three, dtype: int32

In [51]:
data[["three"]][mask]

,three
Colorado,6
Utah,10
New_York,14


In [55]:
# 필터링을 이용한 항목 업데이트
data[~mask] = 0 
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,4,5,6,7
Utah,8,9,10,11
New_York,12,13,14,15


#### 행 단위 인덱싱/슬라이싱

`loc()` 또는 `iloc()` 메서드를 이용한다.

- `loc()` 메서드: 라벨을 이용할 경우
- `iloc()` 메서드: 정수 인덱스를 이용할 경우

In [56]:
data.loc["Colorado"]

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int32

In [58]:
data["one"]

Ohio         0
Colorado     4
Utah         8
New_York    12
Name: one, dtype: int32

In [60]:
data.loc[["Colorado"],["one", "three"]]

,one,three
Colorado,4,6


In [66]:
data.iloc[[1], [1,2]]

,two,three
Colorado,5,6


### 5.2.5 산술 연산

시리즈/데이터프레임의 사칙 연산은 기본적으로 아래 원칙을 따른다.

- 연산에 사용된 모든 인덱스는 포함
- 공통으로 사용되는 인덱스의 항목에 대해서만 연산 적용. 그렇지 않으면 `NaN`으로 처리.

In [67]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

In [68]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [69]:
s1  + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [70]:
s1 / s2

a   -3.476190
c   -0.694444
d         NaN
e   -1.000000
f         NaN
g         NaN
dtype: float64

In [71]:
s1 * s2

a   -15.33
c    -9.00
d      NaN
e    -2.25
f      NaN
g      NaN
dtype: float64

In [72]:
df_1 = pd.DataFrame(np.arange(9).reshape((3,3)),
                   columns=list("bcd"),
                   index = ["Ohio", "Colorado", "Texas"])
df_2 = pd.DataFrame(np.arange(12).reshape((4,3)),
                   columns=list("bde"),
                   index = ["Ohio", "Utah", "New_York", "Texas"])

In [73]:
df_1 + df_2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
New_York,NaN,NaN,NaN,NaN
Ohio,0.0,NaN,3.0,NaN
Texas,15.0,NaN,18.0,NaN
Utah,NaN,NaN,NaN,NaN


- 기본적으로 사용되는 산술 연산 기호에 해당하는 메서드가 존재한다.

| 메서드 | 설명 |
| :--- | :--- |
| `add()` | 덧셈(`+`) 계산 메서드 | 
| `sub()` | 뺄셈(`-`) 계산 메서드 | 
| `mul()` | 곱셈(`*`) 계산 메서드 | 
| `div()` | 나눗셈(`/`) 계산 메서드 | 
| `floordiv()` | 몫 (`//`) 계산 메서드 | 
| `pow()` | 거듭제곱(`**`) 메서드 | 

#### 연산 과정에서 결측치 채우기
- 공통 인덱스가 아니거나 결측치가 이미 존재하는 경우 기본적으로 결측치로 처리됨.
- `fill_value` 키워드 인자를 이용하여 지정된 값으로 처리하게 만들 수도 있다.

In [75]:
print(df_1 + df_2)
print(df_1)
print(df_2)

             b   c     d   e
Colorado   NaN NaN   NaN NaN
New_York   NaN NaN   NaN NaN
Ohio       0.0 NaN   3.0 NaN
Texas     15.0 NaN  18.0 NaN
Utah       NaN NaN   NaN NaN
          b  c  d
Ohio      0  1  2
Colorado  3  4  5
Texas     6  7  8
          b   d   e
Ohio      0   1   2
Utah      3   4   5
New_York  6   7   8
Texas     9  10  11


In [77]:
# df_1 + df_2
df_1.add(df_2, fill_value = 10)

,b,c,d,e
Colorado,13.0,14.0,15.0,NaN
New_York,16.0,NaN,17.0,18.0
Ohio,0.0,11.0,3.0,12.0
Texas,15.0,17.0,18.0,21.0
Utah,13.0,NaN,14.0,15.0


In [78]:
df_2.add(df_1, fill_value = 10)

,b,c,d,e
Colorado,13.0,14.0,15.0,NaN
New_York,16.0,NaN,17.0,18.0
Ohio,0.0,11.0,3.0,12.0
Texas,15.0,17.0,18.0,21.0
Utah,13.0,NaN,14.0,15.0


In [82]:
df_1.add(df_2, fill_value = 0).fillna(0) #Nan 값에 0을 채워 넣고, 데이터 프레임 둘중 하나만 없는 값은 0 을 채워 넣는다. 

,b,c,d,e
Colorado,3.0,4.0,5.0,0.0
New_York,6.0,0.0,7.0,8.0
Ohio,0.0,1.0,3.0,2.0
Texas,15.0,7.0,18.0,11.0
Utah,3.0,0.0,4.0,5.0


#### 데이터프레임과 시리즈 사이의 연산

넘파이에서 2차원 어레이와 1차원 어레이 사이에
브로드캐스팅이 가능한 경우,
즉, 차원을 맞출 수 있는 경우에 연산이 가능했다.

In [83]:
arr = np.arange(12).reshape((4,3))

In [118]:
arr_1 = np.arange(10,14).reshape(4,1)

In [119]:
arr_1.shape

(4, 1)

In [120]:
arr.shape

(4, 3)

In [121]:
arr_1 + arr # 브로드캐스팅에 의해서 적당하게 차원이 맞는다면 연산이 진행된다. 

array([[10, 11, 12],
       [14, 15, 16],
       [18, 19, 20],
       [22, 23, 24]])

In [122]:
arr_2 = np.arange(10,13).reshape(1,3)

In [123]:
arr + arr_2 # # 브로드캐스팅에 의해서 적당하게 차원이 맞는다면 연산이 진행된다. 

array([[10, 12, 14],
       [13, 15, 17],
       [16, 18, 20],
       [19, 21, 23]])

In [124]:
df_2.shape

(4, 3)

In [125]:
arr.shape

(4, 3)

In [126]:
df_2 + arr

,b,d,e
Ohio,0,2,4
Utah,6,8,10
New_York,12,14,16
Texas,18,20,22


In [127]:
df_2

,b,d,e
Ohio,0,1,2
Utah,3,4,5
New_York,6,7,8
Texas,9,10,11


In [128]:
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [129]:
arr_1.shape

(4, 1)

In [130]:
arr_1

array([[10],
       [11],
       [12],
       [13]])

In [131]:
df_2 + arr_1 # 브로드캐스팅 때문에 가능하다. 

,b,d,e
Ohio,10,11,12
Utah,14,15,16
New_York,18,19,20
Texas,22,23,24


In [135]:
arr_1.shape

(4, 1)

In [132]:
arr_2

array([[10, 11, 12]])

In [133]:
df_2 + arr_2

,b,d,e
Ohio,10,12,14
Utah,13,15,17
New_York,16,18,20
Texas,19,21,23


In [136]:
arr_3 = np.arange(3).reshape((3,1))

In [137]:
arr_3

array([[0],
       [1],
       [2]])

In [140]:
# arr_3 + df_2
# 보통 문제상황은 shape 에서 많이 발생한다. 
# Unable to coerce to DataFrame, shape must be (4, 3): given (3, 1)

### 1.2.6 함수 적용

#### 유니버설 함수

유니버설 함수는 넘파이의 경우와 동일하게 작동한다.

In [141]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.001463,-1.262124,1.043154
Ohio,0.667566,0.423268,0.183097
Texas,-0.313308,1.547979,-0.686574
Oregon,0.186401,-1.544753,-1.029036


- 넘파이의 `abs()` 함수를 적용하면 항목별로 이루어진다.

In [142]:
np.abs(frame)

,b,d,e
Utah,0.001463,1.262124,1.043154
Ohio,0.667566,0.423268,0.183097
Texas,0.313308,1.547979,0.686574
Oregon,0.186401,1.544753,1.029036


In [144]:
np.exp(frame) # e^x 에 적용할 수 있다. 

,b,d,e
Utah,0.998538,0.283052,2.838154
Ohio,1.949487,1.526944,1.200931
Texas,0.731025,4.701958,0.503298
Oregon,1.204906,0.213365,0.357351


- 시리즈에 대해서도 동일하게 동작

In [146]:
np.exp(frame["e"])

Utah      2.838154
Ohio      1.200931
Texas     0.503298
Oregon    0.357351
Name: e, dtype: float64

In [147]:
np.abs(frame["e"])

Utah      1.043154
Ohio      0.183097
Texas     0.686574
Oregon    1.029036
Name: e, dtype: float64

#### `map()`과 `applymap()` 메서드 

In [153]:
# elementwise : 요소곱, 요소합 등의 연산이 적용될때 를 의미 한다. 
pd.DataFrame(np.arange(0,9).reshape(3,3)) + pd.DataFrame(np.arange(10,19).reshape(3,3))

,0,1,2
0,10,12,14
1,16,18,20
2,22,24,26


In [155]:
pd.DataFrame(np.arange(0,9).reshape(3,3))

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [154]:
pd.DataFrame(np.arange(10,19).reshape(3,3))

,0,1,2
0,10,11,12
1,13,14,15
2,16,17,18


In [151]:
frame.map?

Signature: frame.map(func: 'PythonFuncType', na_action: 'str | None' = None, **kwargs) -> 'DataFrame'
Docstring:
Apply a function to a Dataframe elementwise.

.. versionadded:: 2.1.0

   DataFrame.applymap was deprecated and renamed to DataFrame.map.

This method applies a function that accepts and returns a scalar
to every element of a DataFrame.

Parameters
----------
func : callable
    Python function, returns a single value from a single value.
na_action : {None, 'ignore'}, default None
    If 'ignore', propagate NaN values, without passing them to func.
**kwargs
    Additional keyword arguments to pass as keywords arguments to
    `func`.

Returns
-------
DataFrame
    Transformed DataFrame.

See Also
--------
DataFrame.apply : Apply a function along input axis of DataFrame.
DataFrame.replace: Replace values given in `to_replace` with `value`.
Series.map : Apply a function elementwise on a Series.

Examples
--------
>>> df = pd.DataFrame([[1, 2.12], [3.356, 4.567]])
>>> df
      

In [156]:
np.round(1.23423442, 3) # 정해진 소숫점 까지 반올림 하는 함수 

1.234

In [158]:
x = 1.23423442
f"{float(x):.3f}" # 문자가 return 된다. 

'1.234'

In [159]:
format_0 = lambda x: np.round(x, 3)
format_1 = lambda x: float(f"{float(x):.3f}")

In [161]:
frame["e"].map(format_0) #series 의 map

Utah      1.043
Ohio      0.183
Texas    -0.687
Oregon   -1.029
Name: e, dtype: float64

In [162]:
frame["e"].map(format_1)

Utah      1.043
Ohio      0.183
Texas    -0.687
Oregon   -1.029
Name: e, dtype: float64

In [165]:
frame[["e"]]

,e
Utah,1.043154
Ohio,0.183097
Texas,-0.686574
Oregon,-1.029036


In [167]:
frame[["e"]].map(format_1)

,e
Utah,1.043
Ohio,0.183
Texas,-0.687
Oregon,-1.029


In [168]:
frame[["e"]].applymap(format_1)

C:\Users\ohminy\AppData\Local\Temp\ipykernel_19444\3589430907.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame[["e"]].applymap(format_1)


,e
Utah,1.043
Ohio,0.183
Texas,-0.687
Oregon,-1.029


In [169]:
frame.map(format_1)

,b,d,e
Utah,-0.001,-1.262,1.043
Ohio,0.668,0.423,0.183
Texas,-0.313,1.548,-0.687
Oregon,0.186,-1.545,-1.029


In [170]:
frame.apply?

Signature:
frame.apply(
    func: 'AggFuncType',
    axis: 'Axis' = 0,
    raw: 'bool' = False,
    result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None,
    args=(),
    by_row: "Literal[False, 'compat']" = 'compat',
    engine: "Literal['python', 'numba']" = 'python',
    engine_kwargs: 'dict[str, bool] | None' = None,
    **kwargs,
)
Docstring:
Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is
either the DataFrame's index (``axis=0``) or the DataFrame's columns
(``axis=1``). By default (``result_type=None``), the final return type
is inferred from the return type of the applied function. Otherwise,
it depends on the `result_type` argument.

Parameters
----------
func : function
    Function to apply to each column or row.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Axis along which the function is applied:

    * 0 or 'index': apply function to each column.
    * 1 or 'columns': apply function

In [148]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.map.html#pandas.DataFrame.map
# New in version 2.1.0: DataFrame.applymap was deprecated and renamed to DataFrame.map.

아래 표는 `Series.map`, `DataFrame.map`, `Series.apply`, `DataFrame.apply`의 차이점을 정리한 것입니다.

| 기능/함수         | `Series.map`        | `DataFrame.map` | `Series.apply`       | `DataFrame.apply`    |
|-----------------|---------------------|----------------------|----------------------|----------------------|
| **적용 대상**    | Series의 각 요소    | DataFrame의 모든 요소 | Series의 각 요소    | DataFrame의 행 또는 열 |
| **기능**         | 각 요소를 새 값으로 매핑 | 각 셀에 동일 연산 적용  | 각 요소에 함수 적용   | 행/열에 함수 적용       |
| **주요 사용 사례** | 데이터 변환, 치환     | 셀 값 변환             | 복잡한 함수 적용     | 행/열 기반 복잡한 연산  |
| **예시**         | `s.map(lambda x: x * 2)` | `df.applymap(lambda x: x * 2)` | `s.apply(lambda x, n: x * n, n=2)` | `df.apply(lambda x: x.sum(), axis=0)` |

이 표를 통해 각 함수의 주요 사용처와 적용 방식을 이해하고 상황에 맞게 적절한 함수를 선택할 수 있습니다.

- 유니버설 함수가 아닌 함수를 `Series`의 항목별 적용 하기위해선 `map()`를 사용한다

__예__ 
- 람다(lambda) 함수를 이용해 부동소수점을 소수점 이하 셋째 자리에서 반올림한 값만 보여주도록 한다.

- 시리즈에 적용

- 유니버설 함수가 아닌 함수를 `데이터프레임`의 항목별로 적용하려면 `applymap()`를 사용한다.

#### `apply()` 메서드

- DataFrame의 행 또는 열 단위로 함수를 적용하려면 `apply()` 메서드를 활용한다.
- 기본은 열 단위로 함수가 적용되며 반환값이 스칼라 값이면 시리즈가 반환

__예__ 
- 최댓값과 최소값의 차이를 반환하는 함수를 적용해 보자

- 행 별로 함수 적용
    - `axis=1` 또는 `axis='columls'` 지정

In [171]:
def fun_2(x):
    return pd.Series([x.min(), x.max()], index=["min","max"])

In [186]:
# series 를 입력으로 받고 series 을 return 하는 함수를 만들고 apply 해보자
def fun_2(x:pd.Series) -> pd.Series:
    return pd.Series([x.min(), x.max()], index=["min","max"])

In [174]:
ser_1 = pd.Series(range(3), index = list("dsv"))

In [175]:
ser_1.min()

0

In [177]:
ser_1.max()

2

In [178]:
fun_1 = lambda x: x.max() - x.min()

In [179]:
frame

,b,d,e
Utah,-0.001463,-1.262124,1.043154
Ohio,0.667566,0.423268,0.183097
Texas,-0.313308,1.547979,-0.686574
Oregon,0.186401,-1.544753,-1.029036


In [182]:
# frame.apply(fun_1)
frame.apply(fun_1, axis = 0)

b    0.980874
d    3.092732
e    2.072190
dtype: float64

In [183]:
frame.apply(fun_1, axis = 1) # 행별로 함수 적용 , axis = 1,  또는 axis = "columns"

Utah      2.305278
Ohio      0.484469
Texas     2.234553
Oregon    1.731154
dtype: float64

In [187]:
fun_2(ser_1)

min    0
max    2
dtype: int64

In [188]:
frame.apply(fun_2)

,b,d,e
min,-0.313308,-1.544753,-1.029036
max,0.667566,1.547979,1.043154


In [190]:
ser_1

d    0
s    1
v    2
dtype: int64

In [189]:
ser_1.apply(fun_2) #ser_1 의 요소가 int 인데 int 에는min max fun 이 없다. 
# apply 는 ser_1의 행 또는 열 별로, dataframe 에는 행 또는 열별로 

AttributeError: 'int' object has no attribute 'min'

In [ ]:
# map : map 은 dataframe 또는 Series 에 각 요소별로 함수를 적용 한다. 
# apply : apply 는 series 를 입력받아 Dataframe로 출력한다. 


| 기능/함수      | `map`                                              | `apply`                                                     |
|--------------|----------------------------------------------------|-------------------------------------------------------------|
| **적용 대상** | Series, DataFrame의 각 요소에 적용                             | Series 또는 DataFrame의 행/열에 적용                        |
| **기능**      | 각 요소를 새 값으로 매핑                            | 복잡한 함수를 적용하거나, 여러 열/행의 데이터를 처리         |
| **사용처**    | 데이터 변환, 치환                                   | 복잡한 데이터 변환, 여러 열/행의 집계나 변환                |
| **예시**      | `s = pd.Series([1, 2, 3]); s.map(lambda x: x * 2)` | `df = pd.DataFrame([[1, 2], [3, 4]]); df.apply(np.sum, axis=0)` |

### 1.2.7 정렬

- 행과 열의 인덱스 또는 항목을 대상으로 정렬할 수 있다.

#### `sort_index()` 메서드
- 시리즈의 경우 인덱스를 기준으로 정렬

In [191]:
ser_1.sort_index()

d    0
s    1
v    2
dtype: int64

내림차순으로 정렬 : `ascending=False` 키워드 인자 사용

In [192]:
ser_1.sort_index(ascending=False)

v    2
s    1
d    0
dtype: int64

데이터프레임의 경우 행 또는 열의 인덱스를 기준으로 정렬
- 기본은 행의 인데스를 기준으로 정렬
- 열의 인덱스를 기준으로 정렬하려면 `axis=1` 또는 `axis='columns'` 키워드 인자를 사용

In [200]:
df_2 = pd.DataFrame(np.arange(18).reshape((3,6)),
                                        index = ["one", "five","two"],
                                        columns = list("lfjvsd"))

In [201]:
df_2

,l,f,j,v,s,d
one,0,1,2,3,4,5
five,6,7,8,9,10,11
two,12,13,14,15,16,17


In [202]:
df_2.sort_index()

,l,f,j,v,s,d
five,6,7,8,9,10,11
one,0,1,2,3,4,5
two,12,13,14,15,16,17


- 내림차순으로 정렬하려면 `ascending=False` 키워드 인자를 함께 사용한다.

In [203]:
df_2.sort_index(ascending=False)

,l,f,j,v,s,d
two,12,13,14,15,16,17
one,0,1,2,3,4,5
five,6,7,8,9,10,11


In [204]:
df_2.sort_index(axis=1)

,d,f,j,l,s,v
one,5,1,2,0,4,3
five,11,7,8,6,10,9
two,17,13,14,12,16,15


In [207]:
df_2.sort_index(axis="columns")

,d,f,j,l,s,v
one,5,1,2,0,4,3
five,11,7,8,6,10,9
two,17,13,14,12,16,15


In [208]:
df_2.sort_index(axis="columns", ascending=False)

,v,s,l,j,f,d
one,3,4,0,2,1,5
five,9,10,6,8,7,11
two,15,16,12,14,13,17


#### `sort_values()` 메서드
- 지정된 열 또는 행에 속한 값들을 기준으로 정렬할 때 사용
- 데이터프레임의 경우 `by` 키워드 인자를 이용하여 열의 label을 지정

In [209]:
ser_3 = pd.Series([4,2,6,7,-1,-100])

In [212]:
ser_3.sort_values(ascending=True)

5   -100
4     -1
1      2
0      4
2      6
3      7
dtype: int64

In [213]:
ser_3.sort_values(ascending=False)

3      7
2      6
0      4
1      2
4     -1
5   -100
dtype: int64

__참고__ 
- `axis=1`을 이용하여 특정 행의 값을 기준으로 정렬 가능

In [214]:
df_ = pd.DataFrame({"b":[4,6,-2, 1], "a":[0, 1, 0, 3]})

In [216]:
df_.sort_values(by = "b")

,b,a
2,-2,0
3,1,3
0,4,0
1,6,1


In [217]:
df_.sort_values(by = "a")

,b,a
0,4,0
2,-2,0
1,6,1
3,1,3


In [218]:
# a 로 정렬 하고 같은 값인경우 a의 순서대로 정렬한다. 
df_.sort_values(by = ["a", "b"])

,b,a
2,-2,0
0,4,0
1,6,1
3,1,3


In [221]:
df_.sort_values(by=0, axis = 1)

,a,b
0,0,4
1,1,6
2,0,-2
3,3,1


### 1.2.8 Groupby
- 데이터를 특정 기준으로 그룹핑할 때 활용
- groupby()를 사용할 때는 반드시 aggregate 하는 통계함수와 일반적으로 같이 적용
<img src="https://www.w3resource.com/w3r_images/pandas-groupby-split-apply-combine.svg">

In [18]:
df = pd.DataFrame({
    'city': ['busan', 'busan', 'busan', 'busan', 'seoul', 'seoul', 'seoul'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})

- city를 기준으로 price의 평균과 quantity의 평균

- `aggregate()`를 이용해 여러 통계함수를 동시에 사용가능

- 2개 이상의 컬럼으로 그룹핑

In [19]:
df = pd.DataFrame({
    'city': ['busan', 'busan', 'busan', 'busan', 'seoul', 'seoul', 'seoul'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})